# <center>__MÉTODOS NUMÉRICOS__</center>
## <center>__PROJETO DA UNIDADE 2__</center>

#### <center>__ALUNO:__Erickson Azevêdo</center>

<div class="alert alert-block alert-info">
1. INTRODUÇÃO
</div>

No contexto do Processamento de Linguagem Natural (PLN), a modelagem de tópicos é um problema de aprendizado não supervisionado cujo objetivo é encontrar tópicos abstratos em uma coleção de documentos. 

A modelagem de tópicos é uma abordagem utilizada para compreesão e classificação automática de dados em uma variedade de configurações, e talvez a aplicação canônica seja descobrir a estrutura temática em um corpus de documentos. Vários trabalhos fundamentais tanto em aprendizado de máquina quanto em teoria sugeriram um modelo probabilístico para documentos, em que os documentos surgem como uma combinação convexa de um pequeno numero de vetores de tópicos, cada vetor de tópico sendo uma distribuição em palavras.

![Modelagem de topicos](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/09/Screenshot_1.png)

<div class="alert alert-block alert-info">
2. DESCRIÇÃO DO PROBLEMA
</div>

Como apresentado na imagem anterior o problema da modelagem de tópicos consiste em determinar se a partir de um corpo de texto de muitos documentos é possivel encontrar os tópicos abstratos sobre os quais o texto está falando.
Essa abordagem é utilizada em diversas ocasiões como por exemplo para entender feedbacks de usuarios [[1]](https://nkoenig06.github.io/gd-tm-svd.html), descobrir quais tópicos estão sendo relevantes no twitter e varias outras aplicações [[2]](https://towardsdatascience.com/lda-topic-modeling-with-tweets-deff37c0e131).

A Modelagem de Tópicos ajuda a destilar as informações no corpus de texto grande em um certo número de tópicos. Os tópicos são grupos de palavras que são *semelhantes* no contexto e são indicativos das informações na coleção de documentos.

A estrutura geral da Matriz Documento-Termo para um corpus de texto contendo M documentos e N termos ao todo é mostrada abaixo:

![Matriz Documento-Termo](https://hackernoon.com/_next/image?url=https%3A%2F%2Fcdn.hackernoon.com%2Fimages%2FBYWRsHWtmGOUC5N4fwNhMqohMAC3-0693jy9.png&w=1920&q=75)

Vamos analisar a representação matricial:

- D1, D2, ..., DM são os documentos M.
- T1, T2, ..., TN são os N termos

Para preencher a Matriz de Termo de Documento, vamos usar a métrica amplamente utilizada – a pontuação TF-IDF.

<div class="alert alert-block alert-info">
3. MÉTODOS APLICADOS À SOLUÇÃO
</div>

Nessa seção, você descreverá que métodos numéricos usará para solucionar o problema acima, explicando como esses métodos funcionam, para que tipos de problemas eles são úteis e, principalmente, porque são úteis para o problema descrito na seção anterior


###  Equação de pontuação TF-IDF

A pontuação TF-IDF é dada pela seguinte equação:

$$ W_{ij} = TF_{ij} * log (\frac{M}{df_{i}})$$

No qual, 
- $TF_{ij}$ é o número de vezes que o termo  $T_j$ ocorre no documento  $D_i$.
- $df_j$ é o número de documentos que contêm o termo $T_j$

Um termo que ocorre com frequência em um documento específico e raramente em todo o corpus tem uma pontuação IDF mais alta.

### Modelagem de tópicos usando decomposição de valor singular (SVD)

O uso de Singular Value Decomposition (SVD) para modelagem de tópicos é explicado na figura abaixo: 

![SVD](https://hackernoon.com/_next/image?url=https%3A%2F%2Fcdn.hackernoon.com%2Fimages%2FBYWRsHWtmGOUC5N4fwNhMqohMAC3-jtl3jg6.jpeg&w=1920&q=75)

A Decomposição de Valor Singular na Matriz de Termo de Documento D fornece as três matrizes a seguir:

- A matriz vetorial singular esquerda U . Esta matriz é obtida pela decomposição própria da matriz Gram D.D_T — também chamada de matriz de similaridade de documentos. A i,j-ésima entrada da matriz de similaridade de documentos significa quão similar é o documento i ao documento j.
- A matriz de valores singulares S que significam a importância relativa dos tópicos.
- A matriz de vetores singulares à direita V_T , que também é chamada de matriz de tópicos do termo. Os tópicos no texto residem ao longo das linhas desta matriz.

O SVD pode ser pensado como uma caixa preta que opera em sua Matriz de Termo de documento (DTM) e produz 3 matrizes, U, S e V_T. E os tópicos residem ao longo das linhas da matriz V_T. isto é ilustrado na figura abaixo.
![Modelador de tópicos com svd](https://hackernoon.com/_next/image?url=https%3A%2F%2Fcdn.hackernoon.com%2Fimages%2FBYWRsHWtmGOUC5N4fwNhMqohMAC3-s7c3juw.png&w=1920&q=75)

<div class="alert alert-block alert-info">
4. IMPLEMENTAÇÃO
</div>

## 1. Importando dependencias e realizando instalações

### Necessario realizar o download da dependencia caso não possua
- python3 -m spacy download en
- pip install -U spacy


In [330]:
!pip install -U spacy
!pip install pyldavis
!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [331]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

## 2. Realizando a leitura dos dados

In [332]:
df = pd.read_csv('google.csv')
df = df.dropna(subset=['Translated_Review'])
df

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.000000,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.250000,0.288462
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.400000,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.000000,0.300000
5,10 Best Foods for You,Best way,Positive,1.000000,0.300000
...,...,...,...,...,...
64222,Housing-Real Estate & Property,Most ads older many agents ..not much owner po...,Positive,0.173333,0.486667
64223,Housing-Real Estate & Property,"If photos posted portal load, fit purpose. I'm...",Positive,0.225000,0.447222
64226,Housing-Real Estate & Property,"Dumb app, I wanted post property rent give opt...",Negative,-0.287500,0.250000
64227,Housing-Real Estate & Property,I property business got link SMS happy perform...,Positive,0.800000,1.000000


## 3. Limpeza de dados

In [333]:
# Convert to list
data = df.Translated_Review.values.tolist()
# Remove Emails
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
# Remove new line characters
data = [re.sub(r'\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub(r"\'", "", sent) for sent in data]
print(data[:1])

['I like eat delicious food. Thats Im cooking food myself, case "10 Best Foods" helps lot, also "Best Before (Shelf Life)"']


## 4. Tokenizar

Agora queremos tokenizar cada frase em uma lista de palavras , removendo completamente as pontuações e os caracteres desnecessários.

Tokenização é o ato de quebrar uma sequência de strings em pedaços como palavras, palavras-chave, frases, símbolos e outros elementos chamados tokens. Os tokens podem ser palavras individuais, frases ou até mesmo frases inteiras. No processo de tokenização , alguns caracteres como sinais de pontuação são descartados.


In [334]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['like', 'eat', 'delicious', 'food', 'thats', 'im', 'cooking', 'food', 'myself', 'case', 'best', 'foods', 'helps', 'lot', 'also', 'best', 'before', 'shelf', 'life']]


## 5. Streamming
Stemming é o processo de redução de uma palavra ao seu radical de palavra que se afixa a sufixos e prefixos ou às raízes de palavras conhecidas como lema.

A vantagem disso é que conseguimos reduzir o número total de palavras únicas no dicionário. Como resultado, o número de colunas na matriz documento-palavra (criada pelo CountVectorizer na próxima etapa) será mais denso com colunas menores. Você pode esperar que tópicos melhores sejam gerados no final

In [335]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [336]:
# Initialize spacy ‘en’ model, keeping only tagger component (for efficiency)
# Run in terminal: python -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'VERB']) #select noun and verb
print(data_lemmatized[:2])

['eat food s m cook food case food help lot shelf life', 'help eat exercise basis']


## 6. Criando uma matriz

Para utilizar o LDA o mesmo necessita uma matriz de palavras do documento como a entrada principal.

Você pode criar um usando CountVectorizer. No código abaixo, configurei o CountVectorizer para considerar palavras que ocorreram pelo menos 10 vezes (min_df), remover palavras irrelevantes em inglês, converter todas as palavras para minúsculas e uma palavra pode conter números e alfabetos de pelo menos 3 comprimentos para ser qualificado como uma palavra.

In [337]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,
                             stop_words='english',             
                             lowercase=True,                   
                             token_pattern='[a-zA-Z0-9]{3,}')
data_vectorized = vectorizer.fit_transform(data_lemmatized)

## 7. Criando modelo LDA com sklearn

In [338]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=10,               
# Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          
# Random state
                                      batch_size=128,            
# n docs in each learning iter
                                      evaluate_every = -1,       
# compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               
# Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)


Como queremos descobrir os melhores parâmetros, usamos a Alocação de Dirichlet Latente com o algoritmo de Bayes variacional online:

In [342]:
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
 evaluate_every=-1, learning_decay=0.7,
 learning_method='online', learning_offset=10.0,
 max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
 n_components=10, n_jobs=-1, perp_tol=0.1,
 random_state=100, topic_word_prior=None,
 total_samples=1000000.0, verbose=0)

LatentDirichletAllocation(learning_method='online', n_jobs=-1, random_state=100)

## 8. Diagnosticar o desempenho do modelo com perplexidade e probabilidade de log

In [343]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))
# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -2121624.4223942687
Perplexity:  1044.592609481927
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


## 9. Usando GridSearch para determinar o melhor modelo LDA

In [ ]:
# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(data_vectorized)
GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_topics': [10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [ ]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

## 10. Tópico dominante

In [ ]:
# Create Document — Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ['Topic' + str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ['Doc' + str(i) for i in range(len(data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
# Styling
def color_green(val):
 color = 'green' if val > .1 else 'black'
 return 'color: {col}'.format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return 'font-weight: {weight}'.format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

## 11. matriz de probabilidade dos tópicos e tópicos dominantes

In [ ]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

- Pegando top 15 palavras chaves

In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

- Inferindo tópicos de acordo com palavras chave.

In [ ]:
Topics = ["Update Version/Fix Crash Problem","Download/Internet Access","Learn and Share","Card Payment","Notification/Support", 
          "Account Problem", "Device/Design/Password", "Language/Recommend/Screen Size", "Graphic/ Game Design/ Level and Coin", "Photo/Search"]
df_topic_keywords["Topics"]=Topics
df_topic_keywords

## 12. Prevendo tópicos usando LDA

In [ ]:
# Define function to predict topic for a given text document.
nlp = spacy.load('en', disable=['parser', 'ner'])
def predict_topic(text, nlp=nlp):
    global sent_to_words
    global lemmatization
# Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text))
# Step 2: Lemmatize
    mytext_3 = lemmatization(mytext_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
# Step 3: Vectorize transform
    mytext_4 = vectorizer.transform(mytext_3)
# Step 4: LDA Transform
    topic_probability_scores = best_lda_model.transform(mytext_4)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), 1:14].values.tolist()
    
    # Step 5: Infer Topic
    infer_topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), -1]
    
    #topic_guess = df_topic_keywords.iloc[np.argmax(topic_probability_scores), Topics]
    return infer_topic, topic, topic_probability_scores
# Predict the topic
mytext = ["Very Useful in diabetes age 30. I need control sugar. thanks Good deal"]
infer_topic, topic, prob_scores = predict_topic(text = mytext)
print(topic)
print(infer_topic)

In [ ]:
# Define function to predict topic for a given text document.
nlp = spacy.load('en', disable=['parser', 'ner'])
def predict_topic(text, nlp=nlp):
    global sent_to_words
    global lemmatization
# Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text))
# Step 2: Lemmatize
    mytext_3 = lemmatization(mytext_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
# Step 3: Vectorize transform
    mytext_4 = vectorizer.transform(mytext_3)
# Step 4: LDA Transform
    topic_probability_scores = best_lda_model.transform(mytext_4)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), 1:14].values.tolist()
    
    # Step 5: Infer Topic
    infer_topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), -1]
    
    #topic_guess = df_topic_keywords.iloc[np.argmax(topic_probability_scores), Topics]
    return infer_topic, topic, topic_probability_scores
# Predict the topic
mytext = ["Very Useful in diabetes age 30. I need control sugar. thanks Good deal"]
infer_topic, topic, prob_scores = predict_topic(text = mytext)
print(topic)
print(infer_topic)

<div class="alert alert-block alert-info">
5. CASOS DE USO
</div>

## 1. Importando dependencias 

In [ ]:
from sklearn import decomposition
from scipy import linalg
import matplotlib.pyplot as plt
import nltk
import gensim

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
lemma = WordNetLemmatizer()

## 2. Carregando os dados a partir do CSV

In [ ]:
data = pd.read_csv('data.csv')
data

## 3. Processo de Tokenization e Streamming

Agora queremos tokenizar cada frase em uma lista de palavras , removendo completamente as pontuações e os caracteres desnecessários.

Tokenização é o ato de quebrar uma sequência de strings em pedaços como palavras, palavras-chave, frases, símbolos e outros elementos chamados tokens. Os tokens podem ser palavras individuais, frases ou até mesmo frases inteiras. No processo de tokenização , alguns caracteres como sinais de pontuação são descartados.

Stemming é o processo de redução de uma palavra ao seu radical de palavra que se afixa a sufixos e prefixos ou às raízes de palavras conhecidas como lema.

A vantagem disso é que conseguimos reduzir o número total de palavras únicas no dicionário. Como resultado, o número de colunas na matriz documento-palavra (criada pelo CountVectorizer na próxima etapa) será mais denso com colunas menores. Você pode esperar que tópicos melhores sejam gerados no final

In [ ]:
data = data.Title.str.split(" ")
data

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
lemma = WordNetLemmatizer()

In [ ]:
data = data.apply(lambda doc: [lemma.lemmatize(word) for word in doc])
data

In [ ]:
from nltk.corpus import stopwords

In [ ]:
swords = stopwords.words('english')

In [ ]:
data = data.apply(lambda doc: [word for word in doc if word not in swords])
data

In [ ]:
from gensim.corpora.dictionary import Dictionary

- Este módulo implementa o conceito de Dicionário – um mapeamento entre palavras e seus ids inteiros

In [ ]:
dct = Dictionary(data)

In [ ]:
dct.filter_extremes(no_below=5, no_above=0.5)

In [ ]:
len(dct.token2id)

In [ ]:
bow = data.apply(dct.doc2bow)
bow

In [ ]:
from gensim.models import LdaModel

In [ ]:
lda = LdaModel(bow, num_topics=20, id2word=dct)

## 5. Aqui é apresentada a matriz de tópicos com sua respectiva importancia para a frase avaliada

In [ ]:
for i in range(20):
    print("Topic_"+str(i), lda.print_topic(i))
    print("")

<div class="alert alert-block alert-info">
6. REFERÊNCIAS
</div>

- [[1]](https://nkoenig06.github.io/gd-tm-svd.html) *Topic Modeling Company Reviews with SVD*, **N. Koenig** - 06/ December/2019
- [[2]](https://towardsdatascience.com/lda-topic-modeling-with-tweets-deff37c0e131) *LDA Topic modeling with Tweets*, **Rob Zifchak** - 10/ Jul/2020
- [[3]](https://hackernoon.com/advanced-topic-modeling-tutorial-how-to-use-svd-and-nmf-in-python-to-find-topics-in-text) *Advanced Topic Modeling Tutorial: How to Use SVD & NMF in Python*,  **@balapriya_** 16 mar 2022
- [[4]](https://yanlinc.medium.com/how-to-build-a-lda-topic-model-using-from-text-601cdcbfd3a6) *How to generate an LDA Topic Model for Text Analysis*